# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/datasets/data.csv')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков


В двух столбцах есть пропущенные значения, найдите их и заполните медианным значением по столбцу. 

Опишите, какие пропущенные значения вы обнаружили;
    


<div class="alert alert-info">
В данном датасете пропущенные значения находятся в столбцах 'days_employed' и 'total_income':

In [4]:
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Проверьте, какую долю составляют пропущенные значения в каждом из столбцов с пропусками.



<div class="alert alert-info">Доля пропущенных значений составляет 10% от общего количества:

In [6]:
data['days_employed'].isna().mean()*100 

10.099883855981417

Приведите возможные причины появления пропусков в данных.


    

<div class="alert alert-info">
Причин может быть несколько: данные не были внесены, были утеряны (человеческий фактор), заёмщики не захотели предоставлять информацию о доходе и стаже.


В двух столбцах есть пропущенные значения, найдите их и заполните медианным значением по столбцу

In [7]:
days_emp_median = data['days_employed'].median()
days_emp_median

-1203.369528770489

In [8]:
income_median = data['total_income'].median()
income_median

145017.93753253992

In [9]:
# заменим пропуски на медиану:
data['days_employed'] = data['days_employed'].fillna(days_emp_median)
data['total_income'] = data['total_income'].fillna(income_median)
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.
    


<div class="alert alert-info">Среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства. Поэтому используем медиану

### Шаг 2.2 Проверка данных на аномалии и исправления.

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце days_employed. Для реальных данных это нормально. Обработайте значения в столбцах с аномалиями и опишите возможные причины появления таких данных.

In [10]:
data['days_employed'] = data['days_employed'].abs()
data['days_employed'].head(10)

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
5       926.185831
6      2879.202052
7       152.779569
8      6929.865299
9      2188.756445
Name: days_employed, dtype: float64

<div class="alert alert-info">Скорее всего, ошибка была допущена изначально при внесении данных

### Шаг 2.3. Изменение типов данных.

Замените вещественный тип данных в столбце total_income на целочисленный, например, с помощью метода astype()

In [11]:
data['total_income'] = data['total_income'].astype('int')
data['total_income'].head(10)

0    253875
1    112080
2    145885
3    267628
4    158616
5    255763
6    240525
7    135823
8     95856
9    144425
Name: total_income, dtype: int64

### Шаг 2.4. Удаление дубликатов.

Если в данных присутствуют строки-дубликаты, удалите их. Также обработайте неявные дубликаты. Например, в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к одному регистру. Проверьте остальные столбцы.

In [12]:
data.duplicated().sum()

54

In [13]:
data = data.drop_duplicates().reset_index(drop=True)
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21467,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21468,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21469,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


In [14]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [15]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [16]:
data['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

<div class="alert alert-info">Две аномалии в этом столбце: '-1' и '20'. -1 заменим на положительное число, 20 вероятнее всего тоже ошибка, заменим на медиану.

In [17]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, data['children'].median())
data['children'] = data['children'].astype('int')
data['children'].value_counts()

0    14183
1     4856
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

In [18]:
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [19]:
#заменим на нижний регистр
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [20]:
data['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

In [21]:
#заменим XNA на 'unknown'
data['gender'] = data['gender'].replace('XNA', 'unknown')
data['gender'].value_counts()

F          14189
M           7281
unknown        1
Name: gender, dtype: int64

In [22]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [23]:
data['debt'].unique()

array([0, 1])

In [24]:
data['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

<div class="alert alert-info"> Очень много неочевидных дубликатов в этом столбце, которые следует категоризировать, что и будет сделано ниже

После удаления дубликатов сделайте следующее:
поясните, как выбирали метод для поиска и удаления дубликатов в данных;
приведите возможные причины появления дубликатов.

<div class="alert alert-info">
Для поиска дубликатов в столбце 'education' использовал unique(), так как он из-за маленького количества данных в этом фрейме позволяет более наглядно понять что не так со значениями. Для других столбцов ввиду большого объема значений применил value_counts().

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создайте два новых датафрейма, в которых:

1) каждому уникальному значению из education соответствует уникальное значение education_id — в первом;

2) каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

Удалите из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. 

Новые датафреймы — это те самые «словари» (не путайте с одноимённой структурой данных в Python), к которым вы сможете обращаться по идентификатору.

In [25]:
data_education = data.loc[:,['education', 'education_id']]
data_education = data_education.drop_duplicates().reset_index(drop=True).sort_values(by='education_id')

In [26]:
data_family_status = data[['family_status', 'family_status_id']]
data_family = data_family_status.drop_duplicates().reset_index(drop=True).sort_values(by='family_status_id')

In [27]:
data=data.drop(columns= ['family_status', 'education'], axis=1)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создайте столбец total_income_category с категориями:

0–30000 — 'E';

30001–50000 — 'D';

50001–200000 — 'C';

200001–1000000 — 'B';

1000001 и выше — 'A'.

In [28]:
def total_income_category(income):
    if income <= 30000:
        return 'E'
    elif 30001<income<=50000:
        return 'D'
    elif 50001<income<=200000:
        return 'C'
    elif 200001<income<=1000000:
        return 'B'
    else: 
        return 'A'
data['total_income_category'] = data['total_income'].apply(total_income_category)
data.head()
    

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


In [29]:
data['total_income_category'].value_counts()

C    16033
B     5040
D      350
A       26
E       22
Name: total_income_category, dtype: int64

In [30]:
data['total_income_category'].count()

21471

### Шаг 2.7. Категоризация целей кредита.

Создайте функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории:

'операции с автомобилем',

'операции с недвижимостью',

'проведение свадьбы',

'получение образования'.

In [31]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [32]:
def purpose_category(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'образовани' in purpose:
        return 'получение образования'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    else:
        return 'операции с недвижимостью'
data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


In [33]:
data['purpose_category'].value_counts()

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

In [34]:
# проверим суммарное количество значений
len(data['purpose_category'])

21471

### Ответы на вопросы.

##### Есть ли зависимость между количеством детей и возвратом кредита в срок?





In [35]:
#найдем конверсию
children_debt = pd.DataFrame()
children_debt['debtors'] = data.groupby('children')['debt'].count() #количество заемщиков 
children_debt['debts'] = data.groupby('children')['debt'].sum() #количество просрочек
children_debt['conversion'] = (children_debt['debts'] / children_debt['debtors']) #конверсия
children_debt.sort_values(by='conversion')

,debtors,debts,conversion
children,,,
5,9,0,0.000000
0,14183,1071,0.075513
3,330,27,0.081818
1,4856,445,0.091639
2,2052,194,0.094542
4,41,4,0.097561


<div class="alert alert-info"> Прямая зависимость между увеличением количества детей и увеличением просрочек неочевидная, так как у людей с 3-мя детьми меньше просрочек, чем у людей с 1 или 2-мя детьми. Зависимость есть, хоть и небольшая.

##### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
#найдем конверсию
family_status_debt = pd.DataFrame()
family_status_debt['debtors'] = data.groupby('family_status_id')['debt'].count() #количество заемщиков 
family_status_debt['debts'] = data.groupby('family_status_id')['debt'].sum() #количество просрочек
family_status_debt['conversion'] = (family_status_debt['debts'] / family_status_debt['debtors']) #конверсия 

#для удобства добавим описание идентификаторов
marital_debt = data_family_status.merge(family_status_debt, on='family_status_id', how='left')

marital_debt.sort_values(by='conversion').reset_index(drop=True).value_counts()



family_status          family_status_id  debtors  debts  conversion
женат / замужем        0                 12344    931    0.075421      12344
гражданский брак       1                 4163     388    0.093202       4163
не женат / не замужем  4                 2810     274    0.097509       2810
в разводе              3                 1195     85     0.071130       1195
вдовец / вдова         2                 959      63     0.065693        959
dtype: int64

<div class="alert alert-info">Одинокие люди, ранее побывавшие в браке, имеют меньше просрочек, чем люди, в браке не бывавшие никогда

##### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [37]:
#найдем конверсию
income_debt = pd.DataFrame()
income_debt['debtors'] = data.groupby('total_income_category')['debt'].count() #количество заемщиков 
income_debt['debts'] = data.groupby('total_income_category')['debt'].sum() #количество просрочек
income_debt['conversion'] = (income_debt['debts'] / income_debt['debtors']) #конверсия
income_debt.sort_values(by='conversion')

,debtors,debts,conversion
total_income_category,,,
D,350,21,0.060000
B,5040,356,0.070635
A,26,2,0.076923
C,16033,1360,0.084825
E,22,2,0.090909


<div class="alert alert-info">Исходя из таблицы, уровень дохода не влияет на дисциплинированность по возврату кредита. Зависимсоть прослеживается, хоть и небольшая.
    


##### Как разные цели кредита влияют на его возврат в срок?

In [38]:
#найдем конверсию
purpose_debt = pd.DataFrame()
purpose_debt['debtors'] = data.groupby('purpose_category')['debt'].count() #количество заемщиков 
purpose_debt['debts'] = data.groupby('purpose_category')['debt'].sum() #количество просрочек
purpose_debt['conversion'] = (purpose_debt['debts'] / purpose_debt['debtors']) #конверсия 
purpose_debt.sort_values(by='conversion')

,debtors,debts,conversion
purpose_category,,,
операции с недвижимостью,10814,782,0.072314
проведение свадьбы,2335,186,0.079657
получение образования,4014,370,0.092177
операции с автомобилем,4308,403,0.093547


<div class="alert alert-info">Меньше всего просрочек у тех, кто берёт для сделок с недвижимостью, больше всего - у автозаёмщиков. 

## Общий вывод:

<div class="alert alert-info"> Как видно из обработанных данных, чаще всего за кредитом обращаются семьи без детей со средним достатком для приобретения жилья. Вдовы/вдовцы и разведенные чаще соблюдают сроки погашения. Несвоевременные выплаты по кредитам/займам характерны для одиноких людей, не бывавших в браке. Больше всего просрочек по автокредитам. Так же проявляется неявная зависимость между просрочками и бОльшим количеством детей, однако данная гипотеза требует дополнительного исследования. Зависимость просрочек от уровня дохода заёмщика отсутствует. 